# SARIMA Grid Search

In [4]:
# grid search sarima hyperparameters
from math import sqrt
from multiprocessing import cpu_count
from joblib import Parallel
from joblib import delayed
from warnings import catch_warnings
from warnings import filterwarnings
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error

In [2]:
# one-step sarima forecast
def sarima_forecast(history, config):
    order, sorder, trend = config
    # define model
    model = SARIMAX(history, order=order, seasonal_order=sorder, trend=trend, enforce_stationarity=False, enforce_invertibility=False)
    # fit model
    model_fit = model.fit(disp=False)
    # make one step forecast
    yhat = model_fit.predict(len(history), len(history))
    return yhat[0]

# root mean squared error or rmse
def measure_rmse(actual, predicted):
    return sqrt(mean_squared_error(actual, predicted))

# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
    return data[:-n_test], data[-n_test:]

# walk-forward validation for univariate data
def walk_forward_validation(data, n_test, cfg):
    predictions = list()
    # split dataset
    train, test = train_test_split(data, n_test)
    # seed history with training dataset
    history = [x for x in train]
    # step over each time-step in the test set
    for i in range(len(test)):
        # fit model and make forecast for history
        yhat = sarima_forecast(history, cfg)
        # store forecast in list of predictions
        predictions.append(yhat)
        # add actual observation to history for the next loop
        history.append(test[i])
    # estimate prediction error
    error = measure_rmse(test, predictions)
    return error

# score a model, return None on failure
def score_model(data, n_test, cfg, debug=False):
    result = None
    # convert config to a key
    key = str(cfg)
    # show all warnings and fail on exception if debugging
    if debug:
        result = walk_forward_validation(data, n_test, cfg)
    else:
        # one failure during model validation suggests an unstable config
        try:
            # never show warnings when grid searching, too noisy
            with catch_warnings():
                filterwarnings("ignore")
                result = walk_forward_validation(data, n_test, cfg)
        except:
            error = None
    # check for an interesting result
    if result is not None:
        print(' > Model[%s] %.3f' % (key, result))
    return (key, result)

# grid search configs
def grid_search(data, cfg_list, n_test, parallel=True):
    scores = None
    if parallel:
        # execute configs in parallel
        executor = Parallel(n_jobs=cpu_count(), backend='multiprocessing')
        tasks = (delayed(score_model)(data, n_test, cfg) for cfg in cfg_list)
        scores = executor(tasks)
    else:
        scores = [score_model(data, n_test, cfg) for cfg in cfg_list]
    # remove empty results
    scores = [r for r in scores if r[1] != None]
    # sort configs by error, asc
    scores.sort(key=lambda tup: tup[1])
    return scores

# create a set of sarima configs to try
def sarima_configs(seasonal=[0]):
    models = list()
    # define config lists
    p_params = [0, 1, 2]
    d_params = [0, 1]
    q_params = [0, 1, 2]
    t_params = ['n','c','t','ct']
    P_params = [0, 1, 2]
    D_params = [0, 1]
    Q_params = [0, 1, 2]
    m_params = seasonal
    # create config instances
    for p in p_params:
        for d in d_params:
            for q in q_params:
                for t in t_params:
                    for P in P_params:
                        for D in D_params:
                            for Q in Q_params:
                                for m in m_params:
                                    cfg = [(p,d,q), (P,D,Q,m), t]
                                    models.append(cfg)
    return models

In [3]:
# define dataset
data = [10.0, 20.0, 30.0, 40.0, 50.0, 60.0, 70.0, 80.0, 90.0, 100.0]

# data split
n_test = 4
# model configs
cfg_list = sarima_configs()
# grid search
scores = grid_search(data, cfg_list, n_test)
print('done')
# list top 3 configs
for cfg, error in scores[:3]:
    print(cfg, error)

 > Model[[(0, 0, 0), (0, 0, 0, 0), 'n']] 85.732
 > Model[[(0, 0, 0), (0, 0, 0, 0), 'c']] 37.914
 > Model[[(0, 0, 0), (1, 0, 0, 0), 'n']] 6.103
 > Model[[(0, 0, 0), (2, 0, 0, 0), 'n']] 0.000
 > Model[[(0, 0, 0), (2, 0, 1, 0), 'n']] 0.000
 > Model[[(0, 0, 0), (1, 0, 1, 0), 'n']] 3.129
 > Model[[(0, 0, 0), (0, 0, 1, 0), 'n']] 85.732
 > Model[[(0, 0, 0), (0, 0, 0, 0), 't']] 6.103
 > Model[[(0, 0, 0), (1, 0, 0, 0), 't']] 6.102
 > Model[[(0, 0, 1), (0, 0, 0, 0), 'n']] 85.732
 > Model[[(0, 0, 0), (0, 0, 1, 0), 't']] 10.000
 > Model[[(0, 0, 0), (0, 0, 1, 0), 'c']] 42.866
 > Model[[(0, 0, 1), (0, 0, 1, 0), 'n']] 87.892
 > Model[[(0, 0, 1), (0, 0, 1, 0), 'c']] 66.123
 > Model[[(0, 0, 0), (0, 0, 0, 0), 'ct']] 0.040
 > Model[[(0, 0, 0), (1, 0, 0, 0), 'ct']] 1.045
 > Model[[(0, 0, 1), (2, 0, 0, 0), 'n']] 0.000
 > Model[[(0, 0, 1), (1, 0, 0, 0), 'n']] 6.105
 > Model[[(0, 0, 0), (2, 0, 0, 0), 't']] 0.019
 > Model[[(0, 0, 1), (0, 0, 0, 0), 't']] 46.537
 > Model[[(0, 0, 0), (1, 0, 1, 0), 't']] 3.129
 >

 > Model[[(1, 1, 1), (0, 0, 0, 0), 'ct']] 0.000
 > Model[[(1, 1, 1), (1, 0, 0, 0), 't']] 0.016
 > Model[[(1, 1, 1), (0, 0, 1, 0), 'ct']] 0.000
 > Model[[(1, 1, 1), (2, 0, 1, 0), 'c']] 106699514.088
 > Model[[(1, 1, 1), (2, 0, 0, 0), 'c']] 0.000
 > Model[[(1, 1, 1), (1, 0, 1, 0), 'c']] 0.050
 > Model[[(2, 0, 0), (0, 0, 0, 0), 'n']] 0.000
 > Model[[(1, 1, 1), (2, 0, 0, 0), 't']] 0.000
 > Model[[(2, 0, 0), (0, 0, 1, 0), 'n']] 0.000
 > Model[[(2, 0, 0), (2, 0, 0, 0), 'n']] 0.000
 > Model[[(1, 1, 1), (1, 0, 0, 0), 'ct']] 64066092.230
 > Model[[(1, 1, 1), (2, 0, 1, 0), 't']] 0.000
 > Model[[(2, 0, 0), (0, 0, 0, 0), 'c']] 0.000
 > Model[[(2, 0, 0), (2, 0, 1, 0), 'n']] 0.000
 > Model[[(2, 0, 0), (0, 0, 1, 0), 'c']] 0.000
 > Model[[(1, 1, 1), (1, 0, 1, 0), 't']] 0.000
 > Model[[(1, 1, 0), (2, 0, 1, 0), 'ct']] 0.000
 > Model[[(1, 1, 1), (2, 0, 1, 0), 'ct']] 0.049
 > Model[[(1, 1, 1), (2, 0, 0, 0), 'ct']] 0.000
 > Model[[(2, 0, 0), (0, 0, 0, 0), 't']] 0.000
 > Model[[(1, 1, 1), (1, 0, 1, 0), 'ct'

Process ForkPoolWorker-7:
Process ForkPoolWorker-3:
Process ForkPoolWorker-8:
Process ForkPoolWorker-9:
Process ForkPoolWorker-1:
Process ForkPoolWorker-6:
Process ForkPoolWorker-4:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib64/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib64/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib64/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib64/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib64/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib64/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    se

KeyboardInterrupt: 

 > Model[[(2, 1, 0), (0, 0, 1, 0), 'c']] 8205007.230
 > Model[[(2, 0, 1), (2, 0, 0, 0), 'ct']] 0.000
 > Model[[(2, 0, 1), (2, 0, 1, 0), 'ct']] 0.000
